In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:09:35_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter

In [3]:
%load_ext nvcc4jupyter

Source files will be saved in "C:\Users\KXIF\AppData\Local\Temp\tmpwxhby7u3".


# **1) Hello World**

In [4]:
%%cuda
#include<stdio.h>

__global__ void helloCUDA() {
    printf("Hello, CUDA!\n");
}

int main() {
    helloCUDA<<<1, 1>>>();
    cudaDeviceSynchronize();
    return 0;
}

Hello, CUDA!



# **2) Vector Addition**

In [5]:
%%cuda
#include <stdio.h>

#define N 10 // represents the size of the vectors

__global__ void vectorAdd(int *a, int *b, int *c) {
    int tid = threadIdx.x;
    if (tid < N) {
        c[tid] = a[tid] + b[tid];
    }
}

int main() {
    int a[N], b[N], c[N];
    int *d_a, *d_b, *d_c;

    cudaMalloc((void**)&d_a, N * sizeof(int));
    cudaMalloc((void**)&d_b, N * sizeof(int));
    cudaMalloc((void**)&d_c, N * sizeof(int));

    for (int i = 0; i < N; ++i) {
        a[i] = i;
        b[i] = 2 * i;
    }

    cudaMemcpy(d_a, a, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, N * sizeof(int), cudaMemcpyHostToDevice);

    vectorAdd<<<1, N>>>(d_a, d_b, d_c);

    cudaMemcpy(c, d_c, N * sizeof(int), cudaMemcpyDeviceToHost);

    for (int i = 0; i < N; ++i) {
        printf("%d + %d = %d\n", a[i], b[i], c[i]);
    }

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


0 + 0 = 0
1 + 2 = 3
2 + 4 = 6
3 + 6 = 9
4 + 8 = 12
5 + 10 = 15
6 + 12 = 18
7 + 14 = 21
8 + 16 = 24
9 + 18 = 27



# **3) Matrix Multiplication**

In [6]:
%%cuda
#include <stdio.h>

#define N 3

__global__ void matrixMul(int *a, int *b, int *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int sum = 0;

    if (row < N && col < N) {
        for (int k = 0; k < N; k++) {
            sum += a[row * N + k] * b[k * N + col];
        }
        c[row * N + col] = sum;
    }
}

int main() {
    int a[N][N], b[N][N], c[N][N];
    int *d_a, *d_b, *d_c;

    // Allocate device memory
    cudaMalloc((void**)&d_a, N * N * sizeof(int));
    cudaMalloc((void**)&d_b, N * N * sizeof(int));
    cudaMalloc((void**)&d_c, N * N * sizeof(int));

    // Initialize host matrices
    for (int i = 0; i < N * N; ++i) {
        a[i / N][i % N] = i;
        b[i / N][i % N] = 2 * i;
    }

    // Copy host matrices to device
    cudaMemcpy(d_a, a, N * N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, N * N * sizeof(int), cudaMemcpyHostToDevice);

    // Set up grid and block dimensions
    dim3 gridDim(1, 1, 1);
    dim3 blockDim(N, N, 1);

    // Launch the matrixMul kernel with specified grid and block dimensions
    matrixMul<<<gridDim, blockDim>>>(d_a, d_b, d_c);

    // Copy result from device to host
    cudaMemcpy(c, d_c, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    // Print result
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%d\t", c[i][j]);
        }
        printf("\n");
    }

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

30	36	42	
84	108	132	
138	180	222	



# **4) Matrix Addition**

In [7]:
%%cuda
#include <stdio.h>

#define N 3

__global__ void matrixAdd(int *a, int *b, int *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        c[row * N + col] = a[row * N + col] + b[row * N + col];
    }
}

int main() {
    int a[N][N], b[N][N], c[N][N];
    int *d_a, *d_b, *d_c;

    // Allocate device memory
    cudaMalloc((void**)&d_a, N * N * sizeof(int));
    cudaMalloc((void**)&d_b, N * N * sizeof(int));
    cudaMalloc((void**)&d_c, N * N * sizeof(int));

    // Initialize host matrices
    for (int i = 0; i < N * N; ++i) {
        a[i / N][i % N] = i;
        b[i / N][i % N] = 2 * i;
    }

    // Copy host matrices to device
    cudaMemcpy(d_a, a, N * N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, N * N * sizeof(int), cudaMemcpyHostToDevice);

    // Set up grid and block dimensions
    dim3 gridDim(1, 1, 1);
    dim3 blockDim(N, N, 1);

    // Launch the matrixAdd kernel with specified grid and block dimensions
    matrixAdd<<<gridDim, blockDim>>>(d_a, d_b, d_c);

    // Copy result from device to host
    cudaMemcpy(c, d_c, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    // Print result
    printf("Matrix A:\n");
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%d\t", a[i][j]);
        }
        printf("\n");
    }

    printf("\nMatrix B:\n");
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%d\t", b[i][j]);
        }
        printf("\n");
    }

    printf("\nMatrix C (A + B):\n");
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%d\t", c[i][j]);
        }
        printf("\n");
    }

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


Matrix A:
0	1	2	
3	4	5	
6	7	8	

Matrix B:
0	2	4	
6	8	10	
12	14	16	

Matrix C (A + B):
0	3	6	
9	12	15	
18	21	24	



# **5) Matrix Transpose**

In [8]:
%%cuda
#include <stdio.h>

#define N 3

__global__ void matrixTranspose(int *a, int *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        c[col * N + row] = a[row * N + col];
    }
}

int main() {
    int a[N][N], c[N][N];
    int *d_a, *d_c;

    // Allocate device memory
    cudaMalloc((void**)&d_a, N * N * sizeof(int));
    cudaMalloc((void**)&d_c, N * N * sizeof(int));

    // Initialize host matrix
    for (int i = 0; i < N * N; ++i) {
        a[i / N][i % N] = i;
    }

    // Copy host matrix to device
    cudaMemcpy(d_a, a, N * N * sizeof(int), cudaMemcpyHostToDevice);

    // Set up grid and block dimensions
    dim3 gridDim(1, 1, 1);
    dim3 blockDim(N, N, 1);

    // Launch the matrixTranspose kernel with specified grid and block dimensions
    matrixTranspose<<<gridDim, blockDim>>>(d_a, d_c);

    // Copy result from device to host
    cudaMemcpy(c, d_c, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    // Print original matrix
    printf("Original Matrix:\n");
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%d\t", a[i][j]);
        }
        printf("\n");
    }

    // Print transposed matrix
    printf("\nTransposed Matrix:\n");
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%d\t", c[i][j]);
        }
        printf("\n");
    }

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_c);

    return 0;
}

Original Matrix:
0	1	2	
3	4	5	
6	7	8	

Transposed Matrix:
0	3	6	
1	4	7	
2	5	8	



# **6) Open MP**

In [ ]:
%%sh
cat > codeopenmp.c <<EOF
#include<stdio.h>
#include<omp.h>

int main() {
    int thread_id;
    #pragma omp parallel private(thread_id)
    {
        thread_id = omp_get_thread_num();
        printf("Hello, world from thread %d \n", thread_id);
    }
    return 0;
}
EOF

In [12]:
!gcc -fopenmp codeopenmp.c && ./a.out

Hello, world from thread 0 
Hello, world from thread 1 


# **7) MPI**

In [13]:
%%sh
cat > mtc.c << EOF
#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>
#include <math.h>
#include <time.h>

#define MASTER_RANK 0

int main(int argc, char *argv[]) {
    int rank, size;
    long long int total_points = 10000000000;
    long long int points_inside_circle = 0;
    long long int local_points_inside_circle = 0;
    double x, y, distance;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    srand(time(NULL) + rank);

    for (long long int i = 0; i < total_points / size; i++) {
        x = (double)rand() / RAND_MAX;
        y = (double)rand() / RAND_MAX;
        distance = sqrt((x - 0.5) * (x - 0.5) + (y - 0.5) * (y - 0.5));
        if (distance <= 0.5) {
            local_points_inside_circle++;
        }
    }

    // Reduce the local points inside the circle to reduce total count
    MPI_Reduce(&local_points_inside_circle, &points_inside_circle, 1, MPI_LONG_LONG_INT, MPI_SUM, MASTER_RANK, MPI_COMM_WORLD);

    // Below process calculates estimation of Pi value
    if (rank == MASTER_RANK) {
        double pi_estimate = 4.0 * points_inside_circle / total_points;
        printf("Estimated value of Pi: %f\n", pi_estimate);
    }

    MPI_Finalize();
    return 0;
}

In [14]:
!mpicc mtc.c -o mtc -lm && mpirun -n 4 --allow-run-as-root --oversubscribe ./mtc

Estimated value of Pi: 3.141590


#**8) Monte Carlo**

In [10]:
%%cuda
#include <iostream>
#include <limits>
#include <cuda.h>
#include <curand_kernel.h>

using std::cout;
using std::endl;

typedef unsigned long long Count;
typedef std::numeric_limits<double> DblLim;

const Count WARP_SIZE = 32;
const Count NBLOCKS = 640;
const Count ITERATIONS = 1000000;

__global__ void picount(Count *totals) {
    __shared__ Count counter[WARP_SIZE];
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    curandState_t rng;
    curand_init(clock64(), tid, 0, &rng);
    counter[threadIdx.x] = 0;
    for (int i = 0; i < ITERATIONS; i++) {
        float x = curand_uniform(&rng);
        float y = curand_uniform(&rng);
        counter[threadIdx.x] += 1 - int(x * x + y * y);
    }
    if (threadIdx.x == 0) {
        totals[blockIdx.x] = 0;
        for (int i = 0; i < WARP_SIZE; i++) {
            totals[blockIdx.x] += counter[i];
        }
    }
}

int main(int argc, char **argv) {
    int numDev;
    cudaGetDeviceCount(&numDev);
    if (numDev < 1) {
        cout << "CUDA device missing! Do you need to use optirun?\n";
        return 1;
    }
    cout << "Starting simulation with " << NBLOCKS << " blocks, " << WARP_SIZE << " threads, and " << ITERATIONS << " iterations\n";

    Count *hOut, *dOut;
    hOut = new Count[NBLOCKS];
    cudaMalloc(&dOut, sizeof(Count) * NBLOCKS);

    picount<<<NBLOCKS, WARP_SIZE>>>(dOut);

    cudaMemcpy(hOut, dOut, sizeof(Count) * NBLOCKS, cudaMemcpyDeviceToHost);
    cudaFree(dOut);

    Count total = 0;
    for (int i = 0; i < NBLOCKS; i++) {
        total += hOut[i];
    }

    Count tests = NBLOCKS * ITERATIONS * WARP_SIZE;
    cout << "Approximated PI using " << tests << " random tests\n";
    cout.precision(DblLim::max_digits10);
    cout << "PI ~= " << 4.0 * (double)total/(double)tests << endl;

    return 0;
}

Starting simulation with 640 blocks, 32 threads, and 1000000 iterations
Approximated PI using 20480000000 random tests
PI ~= 3.1415979878906248



# **9) FFT**

In [11]:
%%cuda
#include <iostream>
#include <cmath>
#include <cuda_runtime.h>

#define PI 3.14159265358979323846

// Complex number structure
struct Complex {
    float real;
    float imag;
};

// Function to perform butterfly operation in FFT
__device__ void butterfly(Complex& a, Complex& b, float cos_theta, float sin_theta) {
    Complex temp;
    temp.real = b.real * cos_theta - b.imag * sin_theta;
    temp.imag = b.real * sin_theta + b.imag * cos_theta;
    b.real = a.real - temp.real;
    b.imag = a.imag - temp.imag;
    a.real = a.real + temp.real;
    a.imag = a.imag + temp.imag;
}

// Function to perform FFT on a single thread
__global__ void fftKernel(Complex* data, int N) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    // Calculate offset and stride
    int offset = tid % (N / 2);
    int stride = N / 2;
    float cos_theta, sin_theta;
    for (int m = N; m >= 2; m /= 2) {
        for (int k = 0; k < N / m; k++) {
            int j = k * m + offset;
            int i = j + stride;
            cos_theta = cos(-2 * PI * k / N);
            sin_theta = sin(-2 * PI * k / N);
            butterfly(data[j], data[i], cos_theta, sin_theta);
        }
        __syncthreads();
    }
}

// Function to print complex numbers
void printComplexMatrix(Complex* matrix, int rows, int cols) {
    for (int i = 0; i < rows; ++i) {
        for (int j = 0; j < cols; ++j) {
            std::cout << "(" << matrix[i * cols + j].real << ", " << matrix[i * cols + j].imag << ") ";
        }
        std::cout << std::endl;
    }
}

int main() {
    const int N = 8; // Size of the input data (should be a power of 2 for this example)

    // Allocate and initialize host data
    Complex* h_data = new Complex[N * N];
    for (int i = 0; i < N * N; ++i) {
        h_data[i].real = static_cast<float>(i);
        h_data[i].imag = 0.0f;
    }

    // Allocate device data
    Complex* d_data;
    cudaMalloc((void**)&d_data, N * N * sizeof(Complex));

    // Copy data from host to device
    cudaMemcpy(d_data, h_data, N * N * sizeof(Complex), cudaMemcpyHostToDevice);

    // Launch FFT kernel
    dim3 blockSize(N);
    dim3 gridSize(1);
    fftKernel<<<gridSize, blockSize>>>(d_data, N);

    // Copy result back to host
    cudaMemcpy(h_data, d_data, N * N * sizeof(Complex), cudaMemcpyDeviceToHost);

    // Print the result in matrix form
    std::cout << "FFT Result in Matrix Form:" << std::endl;
    printComplexMatrix(h_data, N, N);

    // Cleanup
    cudaFree(d_data);
    delete[] h_data;

    return 0;
}

FFT Result in Matrix Form:
(13.6569, -5.65685) (18.364, -6.36396) (17.4142, 8.58579) (22.1213, 8.87868) (-4.68629, 27.3137) (-2.85786, 29.7279) (7.89949, -30.3848) (10.1924, -35.9203) 
(-19.799, -19.799) (-20.9203, -22.3345) (63.6985, -24.9289) (72.234, -25.3934) (-2.82843, -2.82843) (-2.53553, -3.94975) (14, 0) (15, 0) 
(16, 0) (17, 0) (18, 0) (19, 0) (20, 0) (21, 0) (22, 0) (23, 0) 
(24, 0) (25, 0) (26, 0) (27, 0) (28, 0) (29, 0) (30, 0) (31, 0) 
(32, 0) (33, 0) (34, 0) (35, 0) (36, 0) (37, 0) (38, 0) (39, 0) 
(40, 0) (41, 0) (42, 0) (43, 0) (44, 0) (45, 0) (46, 0) (47, 0) 
(48, 0) (49, 0) (50, 0) (51, 0) (52, 0) (53, 0) (54, 0) (55, 0) 
(56, 0) (57, 0) (58, 0) (59, 0) (60, 0) (61, 0) (62, 0) (63, 0) 



# **10) N Queen**

In [14]:
%%cuda
#include <stdio.h>

#define N 4

__device__ bool isSafe(int board[N][N], int row, int col) {
    int i, j;

    // Check this row on left side
    for (i = 0; i < col; i++)
        if (board[row][i])
            return false;

    // Check upper diagonal on left side
    for (i = row, j = col; i >= 0 && j >= 0; i--, j--)
        if (board[i][j])
            return false;

    // Check lower diagonal on left side
    for (i = row, j = col; j >= 0 && i < N; i++, j--)
        if (board[i][j])
            return false;

    return true;
}

__device__ void solveNQueensKernel(int board[N][N], int col, int* count) {
    // If all queens are placed then count this configuration
    if (col == N) {
        // Print the board
        printf("Solution %d:\n", atomicAdd(count, 1));
        for (int i = 0; i < N; i++) {
            for (int j = 0; j < N; j++)
                printf("%d ", board[i][j]);
            printf("\n");
        }
        printf("\n");
        return;
    }

    // Consider this column and try placing this queen in all rows one by one
    for (int i = 0; i < N; i++) {
        // Check if the queen can be placed on board[i][col]
        if (isSafe(board, i, col)) {
            // Place this queen in board[i][col]
            board[i][col] = 1;
            // Recur to place rest of the queens
            solveNQueensKernel(board, col + 1, count);
            // Backtrack
            board[i][col] = 0;
        }
    }
}

__global__ void solveNQueens(int* count) {
    int board[N][N] = {{0}};
    solveNQueensKernel(board, 0, count);
}

int main() {
    int* d_count;
    cudaMalloc((void**)&d_count, sizeof(int));
    int h_count = 0;
    cudaMemcpy(d_count, &h_count, sizeof(int), cudaMemcpyHostToDevice);

    solveNQueens<<<1, 1>>>(d_count);

    cudaMemcpy(&h_count, d_count, sizeof(int), cudaMemcpyDeviceToHost);
    printf("Number of solutions: %d\n", h_count);

    cudaFree(d_count);

    return 0;
}

Solution 0:
0 0 1 0 
1 0 0 0 
0 0 0 1 
0 1 0 0 

Solution 1:
0 1 0 0 
0 0 0 1 
1 0 0 0 
0 0 1 0 

Number of solutions: 2



# **11) Histogram Computation**

In [ ]:
%%cuda

#include <stdio.h>

#define NUM_BINS 256
#define SIZE 1000

__global__ void computeHistogram(const unsigned char *data, int size, int *histogram) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < size) {
        atomicAdd(&histogram[data[tid]], 1);
    }
}

int main() {
    // Host variables
    unsigned char *h_data;
    int *h_histogram;

    // Device variables
    unsigned char *d_data;
    int *d_histogram;

    // Allocate memory on host
    h_data = (unsigned char *)malloc(SIZE * sizeof(unsigned char));
    h_histogram = (int *)malloc(NUM_BINS * sizeof(int));

    // Generate input data
    for (int i = 0; i < SIZE; ++i) {
        h_data[i] = rand() % NUM_BINS; // Random unsigned char values between 0 and 255
    }

    // Allocate memory on device
    cudaMalloc((void **)&d_data, SIZE * sizeof(unsigned char));
    cudaMalloc((void **)&d_histogram, NUM_BINS * sizeof(int));

    // Copy input data from host to device
    cudaMemcpy(d_data, h_data, SIZE * sizeof(unsigned char), cudaMemcpyHostToDevice);
    cudaMemset(d_histogram, 0, NUM_BINS * sizeof(int)); // Initialize histogram on device

    // Kernel configuration
    int blockSize = 256;
    int numBlocks = (SIZE + blockSize - 1) / blockSize;

    // Launch kernel
    computeHistogram<<<numBlocks, blockSize>>>(d_data, SIZE, d_histogram);

    // Copy histogram from device to host
    cudaMemcpy(h_histogram, d_histogram, NUM_BINS * sizeof(int), cudaMemcpyDeviceToHost);

    // Print histogram
    printf("Histogram:\n");
    for (int i = 0; i < NUM_BINS; ++i) {
        printf("%d: %d\n", i, h_histogram[i]);
    }

    // Free memory
    free(h_data);
    free(h_histogram);
    cudaFree(d_data);
    cudaFree(d_histogram);

    return 0;
}

# **12) Partical Simulation**

In [ ]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define NUM_PARTICLES 1000
#define BLOCK_SIZE 256

typedef struct {
    float x, y, z;
    float vx, vy, vz;
} Particle;

void initializeParticles(Particle* particles, int num_particles) {
    for (int i = 0; i < num_particles; i++) {
        particles[i].x = (float)rand() / RAND_MAX;
        particles[i].y = (float)rand() / RAND_MAX;
        particles[i].z = (float)rand() / RAND_MAX;
        particles[i].vx = 0.1 * ((float)rand() / RAND_MAX - 0.5);
        particles[i].vy = 0.1 * ((float)rand() / RAND_MAX - 0.5);
        particles[i].vz = 0.1 * ((float)rand() / RAND_MAX - 0.5);
    }
}

__global__ void updateParticles(Particle* particles, int num_particles) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < num_particles) {
        particles[idx].x += particles[idx].vx;
        particles[idx].y += particles[idx].vy;
        particles[idx].z += particles[idx].vz;
        particles[idx].vx += 0.01;
        particles[idx].vy += 0.01;
        particles[idx].vz += 0.01;
    }
}

int main() {
    Particle* particles, *d_particles;
    particles = (Particle*)malloc(NUM_PARTICLES * sizeof(Particle));
    cudaMalloc(&d_particles, NUM_PARTICLES * sizeof(Particle));

    initializeParticles(particles, NUM_PARTICLES);
    cudaMemcpy(d_particles, particles, NUM_PARTICLES * sizeof(Particle), cudaMemcpyHostToDevice);

    printf("Initial particle positions:\n");
    for (int i = 0; i < NUM_PARTICLES; i++) {
        printf("Particle %d: (%f, %f, %f)\n", i, particles[i].x, particles[i].y, particles[i].z);
    }

    updateParticles<<<(NUM_PARTICLES + BLOCK_SIZE - 1) / BLOCK_SIZE, BLOCK_SIZE>>>(d_particles, NUM_PARTICLES);

    cudaMemcpy(particles, d_particles, NUM_PARTICLES * sizeof(Particle), cudaMemcpyDeviceToHost);

    printf("\nUpdated particle positions:\n");
    for (int i = 0; i < NUM_PARTICLES; i++) {
        printf("Particle %d: (%f, %f, %f)\n", i, particles[i].x, particles[i].y, particles[i].z);
    }

    free(particles);
    cudaFree(d_particles);

    return 0;
}

# **13) Merge Sort**

In [18]:
%%cuda
#include <stdio.h>
#include <stdlib.h>

#define THREADS_PER_BLOCK 256

__device__ void merge(int* arr, int l, int m, int r) {
    int n1 = m - l + 1;
    int n2 = r - m;

    int* L = (int*)malloc(n1 * sizeof(int));
    int* R = (int*)malloc(n2 * sizeof(int));

    for (int i = 0; i < n1; i++)
        L[i] = arr[l + i];
    for (int j = 0; j < n2; j++)
        R[j] = arr[m + 1 + j];

    int i = 0, j = 0, k = l;
    while (i < n1 && j < n2) {
        if (L[i] <= R[j]) {
            arr[k] = L[i];
            i++;
        } else {
            arr[k] = R[j];
            j++;
        }
        k++;
    }

    while (i < n1) {
        arr[k] = L[i];
        i++;
        k++;
    }

    while (j < n2) {
        arr[k] = R[j];
        j++;
        k++;
    }

    free(L);
    free(R);
}

__global__ void mergeSortKernel(int* arr, int n) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int blockSize = blockDim.x;
    int start = tid * n / (blockSize * gridDim.x);
    int end = (tid + 1) * n / (blockSize * gridDim.x) - 1;

    for (int size = 2; size <= n; size *= 2) {
        for (int subSize = size / 2; subSize > 0; subSize /= 2) {
            __syncthreads();

            int step = size / subSize;
            int k = tid / step * step;

            if (k < n && k + subSize < n && k / blockSize == (k + subSize) / blockSize) {
                merge(arr, k, k + subSize - 1, k + size - 1);
            }
        }
    }
}

void mergeSort(int* arr, int n) {
    int* d_arr;
    cudaMalloc(&d_arr, n * sizeof(int));
    cudaMemcpy(d_arr, arr, n * sizeof(int), cudaMemcpyHostToDevice);

    int blocks = (n + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
    mergeSortKernel<<<blocks, THREADS_PER_BLOCK>>>(d_arr, n);

    cudaMemcpy(arr, d_arr, n * sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(d_arr);
}

int main() {
    int n = 10;
    int* arr = (int*)malloc(n * sizeof(int));

    printf("Original array:\n");
    for (int i = 0; i < n; i++) {
        arr[i] = rand() % 100;
        printf("%d ", arr[i]);
    }
    printf("\n");

    mergeSort(arr, n);

    printf("Sorted array:\n");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    free(arr);

    return 0;
}

Original array:
41 67 34 0 69 24 78 58 62 64 
Sorted array:
0 34 34 41 67 69 69 78 64 78 



# **14) Quick Sort**

In [34]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>
__device__ void swap(int &a, int &b) {
    int temp = a;
    a = b;
    b = temp;
}

__device__ int partition(int *arr, int low, int high) {
    int pivot = arr[high];
    int i = low - 1;
    for (int j = low; j < high; j++) {
        if (arr[j] < pivot) {
            i++;
            swap(arr[i], arr[j]);
        }
    }
    swap(arr[i + 1], arr[high]);
    return i + 1;
}

// Recursive quickSort function
__device__ void quickSortRecursive(int *arr, int low, int high) {
    if (low < high) {
        int pi = partition(arr, low, high);
        quickSortRecursive(arr, low, pi - 1);
        quickSortRecursive(arr, pi + 1, high);
    }
}

__global__ void quickSort(int *arr, int low, int high) {
    if (low < high) {
        int pi = partition(arr, low, high);
        // Call the recursive function
        quickSortRecursive(arr, low, pi - 1);
        quickSortRecursive(arr, pi + 1, high);
    }
}

int main() {
    const int size = 8;
    int h_arr[size] = {12, 11, 13, 5, 6, 7, 1, 10};
    int *d_arr;
    cudaMalloc((void**)&d_arr, size * sizeof(int));
    cudaMemcpy(d_arr, h_arr, size * sizeof(int), cudaMemcpyHostToDevice);
    quickSort<<<1, 1>>>(d_arr, 0, size - 1);
    cudaDeviceSynchronize();
    cudaMemcpy(h_arr, d_arr, size * sizeof(int), cudaMemcpyDeviceToHost);
    std::cout << "Sorted array: ";
    for (int i = 0; i < size; i++) {
        std::cout << h_arr[i] << " ";
    }
    std::cout << std::endl;
    cudaFree(d_arr);
    return 0;
}

Sorted array: 1 5 6 7 10 11 12 13 



# **15) Radix Sort**

In [43]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>

#define BLOCK_SIZE 256

__device__ int getMax(int arr[], int n) {
    int max = arr[0];
    for (int i = 1; i < n; i++) {
        if (arr[i] > max)
            max = arr[i];
    }
    return max;
}

__device__ void countSort(int arr[], int n, int exp) {
    int *output = new int[n];
    int count[10] = {0};

    for (int i = 0; i < n; i++)
        count[(arr[i] / exp) % 10]++;

    for (int i = 1; i < 10; i++)
        count[i] += count[i - 1];

    for (int i = n - 1; i >= 0; i--) {
        output[count[(arr[i] / exp) % 10] - 1] = arr[i];
        count[(arr[i] / exp) % 10]--;
    }

    for (int i = 0; i < n; i++)
        arr[i] = output[i];

    delete[] output;
}

__global__ void radixSort(int arr[], int n) {
    int max = getMax(arr, n);

    for (int exp = 1; max / exp > 0; exp *= 10)
        countSort(arr, n, exp);
}

int main() {
    const int size = 8;
    int h_arr[size] = {170, 45, 75, 90, 802, 24, 2, 66};
    int *d_arr;
    cudaMalloc((void**)&d_arr, size * sizeof(int));
    cudaMemcpy(d_arr, h_arr, size * sizeof(int), cudaMemcpyHostToDevice);
    radixSort<<<1, 1>>>(d_arr, size);
    cudaMemcpy(h_arr, d_arr, size * sizeof(int), cudaMemcpyDeviceToHost);
    std::cout << "Sorted array: ";
    for (int i = 0; i < size; i++) {
        std::cout << h_arr[i] << " ";
    }
    std::cout << std::endl;
    cudaFree(d_arr);
    return 0;
}

Sorted array: 2 24 45 66 75 90 170 802 

